In [7]:
import pandas as pd


In [8]:
factors_path = "data/cancer_factors/"
factors = [
    "eso_factors.csv",
    "kidney_factors.csv",
    "liver_factors.csv",
    "lung_factors.csv",
    "pancreatic_factors.csv",
    "prostate_factors.csv",
    "skin_factors.csv"
]

lat_lon = pd.read_csv("data/us_county_latlng.csv")

In [9]:
def add_lat_lon(factor):
    factor = factor.rename(columns={"CountyFIPS": "fips_code"})
    factor["fips_code"] = factor["fips_code"].astype(int).astype(str).str.zfill(5)
    lat_lon["fips_code"] = lat_lon["fips_code"].astype(str).str.zfill(5)
    factor = factor.merge(lat_lon, on="fips_code", how="left")
    return factor

In [10]:
for factor in factors:
    factor_path = factors_path + factor
    factor = pd.read_csv(factor_path)
    df = add_lat_lon(factor)

In [11]:
# Combine values from each cancer df into a single DataFrame
combined_df = pd.DataFrame()
for factor in factors:
    factor_name = factor.split("_")[0]
    factor_path = factors_path + factor
    df = pd.read_csv(factor_path)
    df = df.rename(columns={"Value": factor_name})
    # Merge with combined_df on fips_code
    if combined_df.empty:
        combined_df = df[["CountyFIPS", factor_name]]
    else:
        combined_df = combined_df.merge(df[["CountyFIPS", factor_name]], on="CountyFIPS", how="outer")

# Save the combined DataFrame to a new CSV file
combined_df.to_csv("data/cancer_factors_combined.csv", index=False)
# Add latitude and longitude to the combined DataFrame
combined_df = pd.read_csv("data/cancer_factors_combined.csv")
combined_df = add_lat_lon(combined_df)
# Save the final combined DataFrame with lat/lon to a new CSV file
combined_df.to_csv("data/cancer_factors_combined.csv", index=False)
combined_df

,fips_code,eso,kidney,liver,lung,pancreatic,prostate,skin,name,lng,lat
0,01001,4.23,19.17,7.98,60.6,14.25,151.69,23.38,Autauga,-86.642900,32.535142
1,01003,5.09,19.47,10.19,67.1,13.82,102.13,27.49,Baldwin,-87.722745,30.727825
2,01005,4.04,17.89,7.71,65.5,14.77,128.66,22.13,Barbour,-85.391068,31.870090
3,01007,3.71,17.38,8.13,55.8,12.70,151.16,16.99,Bibb,-87.126814,32.998376
4,01009,4.40,16.71,9.38,64.9,13.08,134.85,19.94,Blount,-86.567006,33.980871
...,...,...,...,...,...,...,...,...,...,...,...
2679,56037,4.24,13.01,5.40,34.0,9.89,117.80,31.39,Sweetwater,-108.879388,41.659762
2680,56039,3.42,12.87,5.82,31.8,11.25,124.15,35.17,Teton,-110.589698,43.934564
2681,56041,4.81,12.48,5.31,32.0,9.68,128.79,38.80,Uinta,-110.547405,41.287497
2682,56043,3.07,15.03,5.78,45.1,9.92,106.89,24.53,Washakie,-107.682315,43.904378


In [19]:
ratio_file = "data/cancer_factors_lat_lon/cancer_factor_ratio/"
factor_names = "drinking,obesity,diabetes,heart_disease,poverty,noHealthIns,smoking".split(",")
import numpy as np
# Calculate the ratio of each cancer value to the factor value
def calculate_ratios(df):
    ratio_df = pd.read_csv(df)
    for column in factor_names:
        # Calculate z-score for each factor
        ratio_df[column + "_zscore"] = (ratio_df[column] - ratio_df[column].mean()) / ratio_df[column].std()
        # Calculate ratio of cancer value to factor value
        ratio_df[column + "_ratio"] = ratio_df[column] / ratio_df["Value"]
    # Select only the relevant columns
    ratio_df = ratio_df[["CountyFIPS"] + [col for col in ratio_df.columns if col.endswith("_ratio") or col.endswith("_zscore")]]
    return ratio_df

for factor in factors:
    ratio_df = calculate_ratios(factors_path + factor)
    ratio_df = add_lat_lon(ratio_df)
    ratio_df.to_csv(f"{ratio_file}{factor.split('_')[0]}.csv", index=False)
